In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from lxml import etree
from matplotlib.patches import Rectangle
import numpy as np
import glob

In [2]:
images = glob.glob('../input/imagelocationandsegmentation/图片定位与分割数据集/images/*.jpg')

In [3]:
len(images)

In [4]:
xmls = glob.glob('../input/imagelocationandsegmentation/图片定位与分割数据集/annotations/xmls/*.xml')

In [5]:
len(xmls)

In [6]:
xmls_names = [x.split('/')[-1].split('.xml')[0] for x in xmls]

In [7]:
len(xmls_names)

In [8]:
images_names = [x.split('/')[-1].split('.jpg')[0] for x in images]

In [9]:
images_names[:3]

In [10]:
names = list(set(images_names)&set(xmls_names))

In [11]:
len(names)

In [12]:
imgs = [img for img in images if img.split('/')[-1].split('.jpg')[0] in names]

In [13]:
len(imgs)

In [14]:
imgs[:3]

In [15]:
imgs[-3:]

In [16]:
imgs.sort(key=lambda x: x.split('\\')[-1].split('.jpg')[0])

In [17]:
xmls.sort(key=lambda x: x.split('\\')[-1].split('.xml')[0])

In [18]:
scal = 224

In [19]:
def to_labels(path):
    xml = open(r'{}'.format(path)).read()
    sel = etree.HTML(xml)
    width = int(sel.xpath('//size/width/text()')[0])
    height = int(sel.xpath('//size/height/text()')[0])
    xmin = int(sel.xpath('//bndbox/xmin/text()')[0])
    ymin = int(sel.xpath('//bndbox/ymin/text()')[0])
    xmax = int(sel.xpath('//bndbox/xmax/text()')[0])
    ymax = int(sel.xpath('//bndbox/ymax/text()')[0])
    return [xmin/width, ymin/height, xmax/width, ymax/height]

In [20]:
labels = [to_labels(path) for path in xmls]

In [21]:
out1_label, out2_label, out3_label, out4_label = list(zip(*labels))

In [22]:
len(out1_label), len(out2_label), len(out3_label), len(out4_label)

In [23]:
index = np.random.permutation(len(imgs))

In [24]:
images = np.array(imgs)[index]


In [25]:
out1_label = np.array(out1_label)[index]
out2_label = np.array(out2_label)[index]
out3_label = np.array(out3_label)[index]
out4_label = np.array(out4_label)[index]

In [26]:
label_datset = tf.data.Dataset.from_tensor_slices((
                                              out1_label, 
                                              out2_label, 
                                              out3_label, 
                                              out4_label))

In [27]:
image_dataset = tf.data.Dataset.from_tensor_slices(images)

In [28]:
label_datset

In [29]:
def read_jpg(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    return img

In [30]:
def normalize(input_image):
    input_image = tf.image.resize(input_image, [scal, scal])
    input_image = tf.cast(input_image, tf.float32)/127.5 - 1
    return input_image

In [31]:
@tf.function
def load_image(input_image_path):
    input_image = read_jpg(input_image_path)
    input_image = normalize(input_image)
    return input_image

In [32]:
image_dataset = image_dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [33]:
image_dataset

In [34]:
dataset = tf.data.Dataset.zip((image_dataset, label_datset))

In [35]:
dataset

In [36]:
test_count = int(len(images)*0.2)

In [37]:
test_count

In [38]:
train_count = len(images) - test_count

In [39]:
dataset_train = dataset.skip(test_count)

In [40]:
dataset_test = dataset.take(test_count)

In [41]:
BATCH_SIZE = 8
BUFFER_SIZE = 300
STEPS_PER_EPOCH = train_count // BATCH_SIZE
VALIDATION_STEPS = test_count // BATCH_SIZE

In [42]:
train_dataset = dataset_train.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = dataset_test.batch(BATCH_SIZE)

In [43]:
train_dataset

In [44]:
%matplotlib inline

In [45]:
for img, label in train_dataset.take(1):
    plt.imshow(tf.keras.preprocessing.image.array_to_img(img[0]))
    out1, out2, out3, out4 = label
    xmin, ymin, xmax, ymax = (out1[0]).numpy()*scal, (out2[0]).numpy()*scal, (out3[0]).numpy()*scal, (out4[0]).numpy()*scal
    rect = Rectangle((xmin, ymin), (xmax-xmin), (ymax-ymin), fill=False, color='red')
    ax = plt.gca()
    ax.axes.add_patch(rect)

In [46]:
xception = tf.keras.applications.Xception(weights='imagenet', 
                                          include_top=False,
                                          input_shape=(scal, scal, 3))

In [47]:
inputs = tf.keras.layers.Input(shape=(scal, scal, 3))

x = xception(inputs)

x = tf.keras.layers.GlobalAveragePooling2D()(x)

x = tf.keras.layers.Dense(2048, activation='relu')(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)

out1 = tf.keras.layers.Dense(1)(x)
out2 = tf.keras.layers.Dense(1)(x)
out3 = tf.keras.layers.Dense(1)(x)
out4 = tf.keras.layers.Dense(1)(x)

predictions = [out1, out2, out3, out4]

model = tf.keras.models.Model(inputs=inputs, outputs=predictions)

In [48]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
              loss='mse',
              metrics=['mae'])

In [49]:
EPOCHS = 5

In [50]:
history = model.fit(train_dataset, 
                          epochs=EPOCHS,
                          steps_per_epoch=STEPS_PER_EPOCH,
                          validation_steps=VALIDATION_STEPS,
                          validation_data=test_dataset)

In [51]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(EPOCHS)

plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'bo', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 1])
plt.legend()
plt.show()

In [53]:
model.save('detect_v1.h5')

In [54]:
new_model = tf.keras.models.load_model('./detect_v1.h5')

In [55]:
plt.figure(figsize=(8, 24))
for img, _ in test_dataset.take(1):
    out1, out2, out3, out4 = new_model.predict(img)
    for i in range(6):
        plt.subplot(6, 1, i+1)
        plt.imshow(tf.keras.preprocessing.image.array_to_img(img[i]))
        xmin, ymin, xmax, ymax = out1[i]*224, out2[i]*224, out3[i]*224, out4[i]*224,
        rect = Rectangle((xmin, ymin), (xmax-xmin), (ymax-ymin), fill=False, color='red')
        ax = plt.gca()
        ax.axes.add_patch(rect)